In [1]:
from preprocess import load_dataset

data = load_dataset()

提取特征

In [2]:
from feature import extract_features

features_train = extract_features(data['train'], data['all_uid'])
features_valid = extract_features(data['valid'], data['all_uid'])
print('feature dim:', features_train.shape[1])

User Histories: 100%|██████████| 184937/184937 [00:04<00:00, 40654.96it/s]


feature dim: 20


In [3]:
import numpy as np
import pandas as pd

# 互动数量取对数
def extract_log_scale_targets(df: pd.DataFrame):
    output_columns = ['like_count', 'forward_count', 'comment_count']
    targets = []
    for col in output_columns:
        targets.append(np.log( df[col].to_numpy() + 1 ))
    return np.column_stack(targets)

targets_train = extract_log_scale_targets(data['train'])
targets_valid = extract_log_scale_targets(data['valid'])

线性回归 - 拟合+预测

In [6]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(features_train, targets_train)
predicts = model.predict(features_valid)
predicts.shape

(184937, 3)

In [7]:
from metric import compute_metrics

# 计算指标时恢复对数
def exp_interactions(x: np.ndarray) -> np.ndarray:
    return np.exp(x) - 1
compute_metrics(exp_interactions(predicts), exp_interactions(targets_valid))

{'score': 0.29859590862590896}